# For the first part, I am providing two separate solutions:

## 1) the first solution is to scape the saved html webpage as a file using my own method
## 2) the second solution is to use Pandas

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import geocoder

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


This first part uses a generator (generating function) to return the lines in the html file based on the start line and stop line

The second function strips off the html text

In [2]:
def extract_data_lines(filename,start_text,end_text):
    """
    open 'filename', and yield the lines between
    the line that contains, 'start_text' and the
    line that contains 'end_text'
    """
    started = False
    ended = False
    with open(filename,encoding="utf8") as fh: #this means that after it is done reading the file it will close it automatically.
        for line in fh:
            #print(line)
            if start_text in line:
                started=True
                #print(line)
                yield line
            elif (started):
                if end_text in line:
                    ended=True
                    yield line
                    break #the break gets out of the loop over lines in the file
                elif (not ended):
                    yield line
                    
def rm_html_jnk(var):
    if ('title=' in var):
        ii = var.find('">')
        ij = var.find('</a>')
        var = var[ii+2:ij]
        return var
    else:
        if ('<td>' in var) and ('</td>' in var):
            ii = var.find('<td>')
            ij = var.find('</td>')
            var = var[ii+4:ij]
            return var
        if ('<td>' in var) and ('</td>' not in var):
            ii = var.find('<td>')
            var = var[ii+4:]
            return var.split('\n')[0]

In [3]:
filename = 'List_of_postal_codes_of_Canada_M.html'
start_text = '<td>M1A</td>'
end_text = '</tbody><tfoot></tfoot></table>'

data = {'Postcode': [],'Borough': [], 'Neighborhood': []}
lst = ['Postcode','Borough','Neighborhood']
post = {}
tr = False

counter = 0 #since the html file splits the row into three separate parts for each column

idx = 0     #since there are repeat postalcodes this index variable keeps track of unique one in a dictionary

temp = []   #this list gets reset every time it hits a new row of variables

same = False #variable to say whether or not new postalcode is already in the dictionary

for line in extract_data_lines(filename,start_text,end_text):
    if end_text not in line:
        #print(temp)
        if '<tr>' in line or '</tr>' in line: #these tags mark the beginning/end of the column variables for the row
            counter = 0
            temp = []
            same = False
        else:
            trim_line = rm_html_jnk(line)
            #temp.append(trim_line)
            if(counter == 0):
                temp.append(trim_line)
                if trim_line in post:
                    same = True
                else:
                    post[trim_line]=idx
                    data[lst[counter]].append(trim_line)
                    idx+=1
            elif(counter == 1):
                temp.append(trim_line)
                if not same:
                    data[lst[counter]].append(rm_html_jnk(line))
            elif(counter == 2):
                temp.append(trim_line) 
                #Below I reassign the Neighbourhood variable if the Borough variable is set
                if trim_line =='Not assigned' and temp[1] != 'Not assigned':
                    trim_line = temp[1]
                    print('resetting neighborhood from {} to {}'.format(temp[2],trim_line))
                if same: 
                    #We can just add the neighborhoods to all the others of the same postalcode
                    data[lst[counter]][post[temp[0]]] = data[lst[counter]][post[temp[0]]]+', '+trim_line
                else:
                    data[lst[counter]].append(trim_line)
            counter+=1

df = pd.DataFrame(data)

resetting neighborhood from Not assigned to Queen's Park


### As you can see below, this was properly read in except for the fact that we need to drop the rows with 'Not assigned' in the 'Borough' column

In [4]:
print(df.shape)
df.head()

(180, 3)


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [5]:
df_dropped = df[df['Borough'] != 'Not assigned']

In [6]:
df_dropped.reset_index(drop=True,inplace=True)

In [7]:
df_dropped.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [8]:
df_dropped.shape

(103, 3)

## Now for the method to use Pandas to read in the table from the url using pd.read_url()

In [9]:
weburl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
new_df = pd.read_html(weburl)[0]  #The [0] at the end is necessary to pull the first part of the data read in which is the table

In [10]:
print(new_df.shape)
new_df.head()

(288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
new_df_drop = new_df[new_df['Borough'] != 'Not assigned']
new_df_drop.reset_index(drop=True,inplace=True)

In [12]:
def set_same_borough(x):
    for i in range(len(x['Neighbourhood'])):
        if x['Neighbourhood'][i]=='Not assigned':
            x['Neighbourhood'][i]=x['Borough'][i]
    return x

#I am going to set the same neighborhood as the borough if the neighborhood is not assigned
new_df_drop = set_same_borough(new_df_drop)
print(new_df_drop[new_df_drop['Neighbourhood']=='Not assigned'])

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


C:\Users\KevinDWilk\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
new_df_drop.shape

(211, 3)

In [14]:
new_df_group = new_df_drop.groupby('Postcode')

In [15]:
def join_nghds(x):
    x['Neighbourhood'] = ', '.join(x['Neighbourhood'])
    return x
final_df=new_df_group.apply(join_nghds)

In [16]:
final_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"


In [17]:
final_df.drop_duplicates(subset='Postcode',keep='first',inplace=True)
final_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park


In [18]:
print('Note it is the same as the previous method...YAY!')
final_df.shape

Note it is the same as the previous method...YAY!


(103, 3)

# Now on to the second part

## Remember the goal of this part is to read in the coordinates information and make the table.

In [20]:
lat_coords = []
long_coords = []
for postal_code in final_df['Postcode']:
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    print('Finished for postal code: {}'.format(postal_code))
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    lat_coords.append(latitude)
    long_coords.append(longitude)

final_df['Latitude']=lat_coords
final_df['Longitude']=long_coords

KeyboardInterrupt: 

Since this method doesn't seem to be working, I am going to just load in the datasets and merge them.

In [27]:
coords_df = pd.read_csv('Geospatial_Coordinates.csv')

In [28]:
coords_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
coords_df.shape

(103, 3)

They have the same length (103 rows), so I will assume that it has all the same Postal Codes that I have

In [31]:
coords_df.rename(columns={'Postal Code':'Postcode'},inplace=True)

In [33]:
joined_df = pd.merge(final_df,coords_df, on='Postcode',how='inner')

In [37]:
joined_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
